In [ ]:
from dotenv import load_dotenv

load_dotenv()

In [ ]:
import sounddevice as sd
from scipy.io.wavfile import write
import whisper

# 🔴 Step 1: 마이크로 녹음
fs = 16000  # 샘플링 주파수
duration = 5  # 녹음 시간(초)
print("🎙️ 5초 동안 말씀해주세요...")
recording = sd.rec(int(duration * fs), samplerate=fs, channels=1)
sd.wait()
write("order_audio.wav", fs, recording)
print("✅ 녹음 완료!")

# 🔵 Step 2: Whisper로 텍스트 변환
model = whisper.load_model("base")
result = model.transcribe("order_audio.wav")
user_text = result["text"]
print("📝 음성 인식 결과:", user_text)


In [ ]:
from langchain.prompts import PromptTemplate
from langchain.agents import initialize_agent, AgentType
from langchain.tools import StructuredTool
from gtts import gTTS
import os

# GPT 준비
llm = ChatOpenAI(
    temperature=0.1,  # 창의성 (0.0 ~ 2.0)
    model_name="gpt-4o",  # 모델명
    
)

# 프롬프트로 주문 파싱
prompt = PromptTemplate.from_template("""
너당신은 “서보웨이” 가게의 음성 주문을 처리하는 AI 어시스턴트입니다. 다음 규칙에 따라 고객의 주문을 받아 처리하고, 최종 주문 내역을 명확하게 생성해야 합니다.
가게 메뉴 및 규칙:
단일 메뉴: 저희 가게는 오직 “
서보위치 샌드위치” 한 종류만 판매합니다.
고정 가격: 샌드위치 가격은 5,000원입니다.
기본 재료: 샌드위치에는 기본적으로 `빵`, `햄`, `치즈`, `
양상추`가 들어갑니다.
재료 추가 불가: 어떤 재료도 추가할 수 없습니다. 고객이 재료 추가를 요청하면 불가능하다고 정중히 안내해야 합니다.
재료 제외 가능: 고객은 기본 재료(`빵`, `햄`, `치즈`, `
양상추`) 중에서 원하지 않는 것을 빼달라고 요청할 수 있습니다.
가격 변동 없음: 재료를 제외하더라도 샌드위치 가격은 5,000원으로 동일합니다. 이 점을 고객에게 명확히 안내할 필요는 없지만, 시스템은 인지하고 있어야 합니다.
주문 확인: 주문 내용을 명확히 파악한 후, 고객에게 최종 주문 내역(메뉴 이름, 제외된 재료, 가격)을 확인시켜 주어야 합니다.
""")
structured_input = prompt.format(user_input=user_text)
order_info = llm.predict(structured_input)
print("🤖 GPT 응답:", order_info)

# 메뉴 가격 조회 툴
def get_price(menu_item: str):
    db = {"서보위치":  }
    return f"{menu_item}는 {db.get(menu_item, '정보 없음')}원입니다."

price_tool = StructuredTool.from_function(
    func=get_price,
    name="MenuPriceLookup",
    description="음료 이름을 입력하면 가격을 알려주는 툴"
)

# 에이전트 실행
agent = initialize_agent(
    llm=llm,
    tools=[price_tool],
    agent=AgentType.STRUCTURED_CHAT_ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent_response = agent.run(f"{order_info}에 대한 가격을 알려줘")
print("📦 에이전트 응답:", agent_response)

# 음성 출력
tts = gTTS(agent_response, lang='ko')
tts.save("response.mp3")
os.system("mpg123 response.mp3")  # 리눅스 기준. 윈도우는 playsound 써도 됨

